In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.sampler import Sampler
from inference_toolbox.visualiser import Visualiser
from inference_toolbox.domain import Domain

from data_processing.normaliser import Normaliser
from data_processing.box_gridder import BoxGridder
import os

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
log_dummy_data = pd.read_csv('data/log_dummy_data.csv')
training_data, testing_data = train_test_split(log_dummy_data, test_size=0.2)

In [3]:
inputs = pd.read_csv('results/all_results.csv',header=[0,1,2], index_col=0)
inputs.columns


MultiIndex([('Parameters',        'I_y',     'Initial Value'),
            ('Parameters',        'I_y',             'Prior'),
            ('Parameters',        'I_y',                'Mu'),
            ('Parameters',        'I_y',             'Sigma'),
            ('Parameters',        'I_z',     'Initial Value'),
            ('Parameters',        'I_z',             'Prior'),
            ('Parameters',        'I_z',                'Mu'),
            ('Parameters',        'I_z',             'Sigma'),
            ('Parameters',          'Q',     'Initial Value'),
            ('Parameters',          'Q',             'Prior'),
            ('Parameters',          'Q',                'Mu'),
            ('Parameters',          'Q',             'Sigma'),
            ('Parameters',      'Model',              'Type'),
            ('Parameters',      'Model',                 'H'),
            ('Parameters', 'Likelihood',              'Type'),
            ('Parameters', 'Likelihood',             's

In [4]:
for instance in inputs.index:

    # Initialize parameter series
    params = pd.Series({},dtype='float64')

    print('Generating instance ' +str(instance) + '...')

    # Parameter Assignment
    I_y = Parameter(name = 'I_y', init_val=inputs[('Parameters','I_y','Initial Value')][instance], prior_select=inputs[('Parameters','I_y','Prior')][instance].lower())
    I_y.add_prior_param("mu",inputs[('Parameters','I_y','Mu')][instance])
    I_y.add_prior_param("sigma",inputs[('Parameters','I_y','Sigma')][instance])
    params['I_y'] = I_y

    I_z = Parameter(name = 'I_z', init_val=inputs[('Parameters','I_z','Initial Value')][instance], prior_select=inputs[('Parameters','I_z','Prior')][instance].lower())
    I_z.add_prior_param("mu",inputs[('Parameters','I_z','Mu')][instance])
    I_z.add_prior_param("sigma",inputs[('Parameters','I_z','Sigma')][instance])
    params['I_z'] = I_z

    Q = Parameter(name = 'Q', init_val=inputs[('Parameters','Q','Initial Value')][instance], prior_select=inputs[('Parameters','Q','Prior')][instance].lower())
    Q.add_prior_param("mu",inputs[('Parameters','Q','Mu')][instance])
    Q.add_prior_param("sigma",inputs[('Parameters','Q','Sigma')][instance])
    params['Q'] = Q

    # Model Assignment
    model = Model(inputs[('Parameters','Model','Type')][instance].lower())
    model.add_model_param("H",inputs[('Parameters','Model','H')][instance])

    # Likelihood function assigmnent
    likelihood = Likelihood(inputs[('Parameters','Likelihood','Type')][instance].lower())
    likelihood.add_likelihood_param("sigma",inputs[('Parameters','Likelihood','sigma')][instance])
    # likelihood.add_likelihood_param("lambda_1",1)
    # likelihood.add_likelihood_param("lambda_2",0.05)

    num_samples = int(inputs[('Parameters','Sampler','N_samples')][instance])
    num_chains = int(inputs[('Parameters','Sampler','N_chains')][instance])
    thinning_rate_= int(inputs[('Parameters','Sampler','Thinning rate')][instance])


    # Initialize the sampler
    sampler = Sampler(params, model, likelihood, training_data, num_samples, show_sample_info = True, n_chains=num_chains, thinning_rate=thinning_rate_)
    hyperparams = sampler.get_hyperparams()
    params_samples, chain_samples, fields = sampler.sample_all()
    visualiser = Visualiser(testing_data, params_samples, model, sampler.hyperparams, chain_samples=chain_samples,fields = fields, previous_instance = sampler.instance, data_path = 'results/inference')
    summary = visualiser.get_summary()
    visualiser.get_traceplot()

    inputs.loc[instance,('Results','I_y', 'Lower')] = summary['overall']['I_y']['lower']
    inputs.loc[instance,('Results','I_y', 'Mean')] = summary['overall']['I_y']['mean']
    inputs.loc[instance,('Results','I_y', 'Upper')] = summary['overall']['I_y']['upper']
    inputs.loc[instance,('Results','I_y', 'Average tau')] = summary['overall']['I_y']['tau']

    inputs.loc[instance,('Results','I_z', 'Lower')] = summary['overall']['I_z']['lower']
    inputs.loc[instance,('Results','I_z', 'Mean')] = summary['overall']['I_z']['mean']
    inputs.loc[instance,('Results','I_z', 'Upper')] = summary['overall']['I_z']['upper']
    inputs.loc[instance,('Results','I_z', 'Average tau')] = summary['overall']['I_z']['tau']

    inputs.loc[instance,('Results','Q', 'Lower')] = summary['overall']['Q']['lower']
    inputs.loc[instance,('Results','Q', 'Mean')] = summary['overall']['Q']['mean']
    inputs.loc[instance,('Results','Q', 'Upper')] = summary['overall']['Q']['upper']
    inputs.loc[instance,('Results','Q', 'Average tau')] = summary['overall']['Q']['tau']

    inputs.loc[instance,('Results','Misc', 'RMSE')] = summary['RMSE']

    divergences = []
    for i in range(visualiser.num_chains):
        divergences.append(summary['chain_' + str(i+1)]['fields']['diverging'])

    inputs.loc[instance,('Results','Misc', 'Average Diverging')] = np.mean(divergences)

inputs.to_csv('results/all_results.xlsx')

# Sample the parameters
# params_samples, chain_samples, fields = sampler.sample_all()

Generating instance 1...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
